# Importing

In [3]:
import numpy as np
import pandas as pd
import glob
import os


from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN

from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

import keras
from keras.utils import np_utils
from keras.models import Sequential

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\s_NathaLee87665\AppData\Local\Continuum\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\s_NathaLee87665\AppData\Local\Continuum\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\s_NathaLee87665\AppData\Local\Continuum\anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\s_NathaLee87665\AppData\Local\Continuum\anaconda3\envs\tensorflow2\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\s_NathaLee87665\AppData\Local\Continuum\anac

# Open Files

In [6]:
text = (open("training_data.txt", encoding="utf-8").read())
text = text.lower()

# Enumerate Characters

In [7]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

# Vectorize

In [8]:
x = []
y = []

length = len(text)
seq_length = 100

for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    x.append([char_to_n[char] for char in sequence])
    y.append(char_to_n[label])

#shape input vectors to fit conv network
X_modified = np.reshape(x, (len(x), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(y)

# Set up Model

In [13]:
model =  Sequential()

model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), 
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Add Model Checkpoints

In [14]:
modelcheckpoint_ = ModelCheckpoint(filepath='.\models\weights.{epoch:02d}--{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
EarlyStopping_ = EarlyStopping(monitor='val_loss', min_delta=0.06, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
#tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')



# Predict and Generate Text

In [16]:
class PredictModel(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        string_mapped = x[99]
        # generating characters
        string_mapped = x[99]
        full_string = [n_to_char[value] for value in string_mapped]

        for i in range(400):
            x = np.reshape(string_mapped,(1,len(string_mapped), 1))
            x = x / float(len(characters))

            pred_index = np.argmax(model.predict(x, verbose=2))
            seq = [n_to_char[value] for value in string_mapped]
            full_string.append(n_to_char[pred_index])

            string_mapped.append(pred_index)
            string_mapped = string_mapped[1:len(string_mapped)]
        txt=""
        for char in full_string:
            txt = txt+char

        print(txt)

    

# Train Model

In [ ]:
model.fit(X_modified, Y_modified, epochs=100, batch_size=50)
#callbacks=[modelcheckpoint_, EarlyStopping_, PredictModel]

Epoch 1/100
77688/77688 [==============================] - 123s 2ms/step - loss: 2.9778
Epoch 2/100
14300/77688 [====>.........................] - ETA: 1:38 - loss: 2.9645

# Load the weights

In [ ]:
list_of_files = glob.glob('\models\.hdf5') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

model.load_weights(latest_file)

# Display text